# Julia Fundamentals

- Syntax and some simple I/O
- Variables and Scope, Local and Global, Static and Dynamic, and Constants
- Primitive, Abstract and Composite Types
- Operators and elementary math functions
- Strings (Unicode and UTF-8)
- String operations and pattern matching

## 1. Syntax and Simple I/O

- Julia syntax and expressions that are simple and clear
- Terminal I/O
- Simple file I/O using csv files


#### Shell Game

In [4]:
# Shell Game example (Coconut shells and 1 pea)
# Shells are variable names
# Under each shell, type with value
# This example shows that variable names are just labels
# They can be made concrete and that will be shown later
# type and data go together as the following will illustrate
a = 3
println("shell_a is ", typeof(a))
b = [1 2; 3 4]
println("shell_b is ", typeof(b))
c = "pea"
println("shell_c is ", typeof(c))

println("Before: $a, $b, $c")

# We can shuffle these around 
a, b, c = b, c, a # Shell game
a, b ,c = b, a , c # Where is the pea

println("After: $a, $b, $c")

shell_a is Int64
shell_b is Array{Int64,2}
shell_c is String
Before: 3, [1 2; 3 4], pea
After: pea, [1 2; 3 4], 3


#### How deep is a well?

In [6]:
# How deep is a well? Use a stopwatch
# dropping a rock and listening for the splash

# depth = 0.5g * td^2     td = time to reach depth (s = 0.5a * t^2)
# depth = vs * tu         tu = time sound take from bottom to top

g = 9.8065    # acceleration due to gravity
vs = 343.0    # m/s velocity of sound

print("drop to splash time: ")    # Seconds from drop to ear
st = readline();    #Arg STDIN assumed if left out
t = parse(Float32, st)    #String to Float32 convertion for t


# solve equations by
# EQ1: .5g*td^2 - vs*tu = 0
# EQ2: td+tu = t => tu=t-td substitute in EQ1 and get
# quadratic .5g*td^2 + vs*td - vs*t =0 in t1

td = (-vs + sqrt(vs^2 + 2.0g*vs*t))/g
tu = t - td
depthu = vs*tu
println("depth $depthu meters\n sound time $tu seconds")

#confirm
depthd = .5g*td^2
println("confirming depth $depthd meters\n fall time $td seconds")

drop to splash time: stdin> 4
depth 70.58719849910099 meters
 sound time 0.20579358163003203 seconds
confirming depth 70.58719849910122 meters
 fall time 3.794206418369968 seconds


#### Shopping list (I/O csv)

In [9]:
file_in = open("files/str1.csv")
file_out = open("files/str2.csv", "w")
cnt = 1


while(!eof(file_in))
    global cnt  # We need to specify cnt as global so we can write to it,
                # else it would be interpreted as a variable inside the while loop
    row = readline(file_in)
    row = "$cnt,"*row*"\n"
    write(file_out, row)
    print(row)
    cnt += 1
end
close(file_in)
close(file_out)
println("Done!")

1,salt
2,pepper
3,milk
4,cheese
5,tomatoes
6,potatoes
Done!


#### Takeaways

- Learned how variable names are just labels
- Learned what Julia syntax looks like
- Learned how to communicate with your terminal/console window
- Learned reading and writing CSV files that can be used with Excel or OpenOffice

## 2. Variables and Scope - Local and Global, Static and Dynamic, and Constants

- Global and local variables
- Static and Dynamic
- Constants

##### Scope global vs local

If we want to write to a variable inside of a loop, we need to specify it as global inside such loop.

In [11]:
i=0             # (global)
while i == 0    # Ok since being read
    global i    # because being written below
    x = π       # (soft local) needs global to be seen outside of while
    i+= 1       # Need the global since being written
    println(i)
    println(x)  # x is local
end


1
π


In [12]:
println(i)
println(x)      # error since x is local in while

1


UndefVarError: UndefVarError: x not defined

#### "Static"

"Static" type state becomes what in C would be called a static type.
Its value persists but is only available inside the let block.
Useful for creating pointers in buffers, deques and queues.

In [13]:
let
    state=0
    global counter
    counter() = state+=1
end
println(counter())
println(counter())

1
2


In [14]:
# Loop indices and externally defined variables local

i = 43 # these will NOT be overwritten by the loop indfex and local l
l = 66

for i=1:3   # this i is "soft" local so won't interfere with global i
    
    j=i+1   # j is also "soft" local to this block and lower
    
    for k=1:2
        local l # "Hard" local so won't interfere with global l=66
        l = k + j + i
        println("l ",l)
    end
end

l 4
l 5
l 6
l 7
l 8
l 9


In [19]:
println(i)  # 1:3 won't override 43
println(l)  # note inner for loop l is hard local
println(j)  # error j soft local not available to higher level

43
66


UndefVarError: UndefVarError: j not defined

#### Constants

Constants are like variables, but they will warn you if you override them.

In [23]:
const m = 43.5

43.5

In [24]:
println(m)

43.5


In [25]:
m = 42   # warning error

ErrorException: invalid redefinition of constant m

In [26]:
# Built in constants
π #(alias:pi)

π = 3.1415926535897...

In [27]:
# CODATA.jlexp(l) package has 350 physics constants

#### Takeaways

- Scope rules for variables
- Static variables for shared pointers and counters
- Constants cannot be overriden

### Primitive, abstract and composite types